In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm
import re

# NER Imports
import spacy
import en_core_web_sm
from fuzzywuzzy import fuzz

In [2]:
df = pd.read_csv('./Data/covid19_articles_20201231.csv')

In [3]:
# date to datetime
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369047 entries, 0 to 369046
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   author      181799 non-null  object        
 1   date        369047 non-null  datetime64[ns]
 2   domain      369047 non-null  object        
 3   title       368962 non-null  object        
 4   url         369047 non-null  object        
 5   content     369047 non-null  object        
 6   topic_area  369047 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 19.7+ MB


In [4]:
df

,author,date,domain,title,url,content,topic_area
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer
...,...,...,...,...,...,...,...
369042,Polly Harrison,2020-12-31,thefintechtimes,A Human Touch Will Be a Competitive Edge After...,https://thefintechtimes.com/53867-2/,Niels Pedersen is a Chartered Accountant and S...,finance
369043,NaN,2020-12-31,marketscreener,Datametrex AI : Announces Deploying NexaSecuri...,https://www.marketscreener.com/quote/stock/DAT...,Datametrex AI Limited (TSXV: DM) (FSE: D4G) (O...,business
369044,Polly Harrison,2020-12-31,thefintechtimes,"US Payments: Smart Pension, Episode Six, PAAY ...",https://thefintechtimes.com/us-payments-smart-...,"This December, The Fintech Times is asking ind...",finance
369045,NaN,2020-12-31,marketscreener,"WESTWATER RESOURCES, INC. : Entry into a Mater...",https://www.marketscreener.com/quote/stock/WES...,Item 1.01Entry into a Material Definitive Agre...,business


### NER

In [5]:
ner_processor = en_core_web_sm.load()

#test = ner_processor(df.iloc[0]['content'])
#pprint(set([(X.text, X.label_) for X in test.ents if X.label_ == 'ORG']))

In [6]:
filepath_companies = './Data/company_names.xlsx'
companies_list = pd.read_excel(filepath_companies, sheet_name = 'All', header = None)
companies_list = list(set(sorted(companies_list[0])))
print('number of companies in companies list: ', len(companies_list))

org_stopwords = pd.read_excel(filepath_companies, sheet_name = 'Org_stopwords', header = None)
org_stopwords = list(set(sorted(org_stopwords[0])))
new_stopwords = []
org_stopwords = org_stopwords + new_stopwords
org_stopwords = sorted([word.lower() for word in org_stopwords])# + [word.title() for word in org_stopwords])
print('number of phrases in org stopwords list: ', len(org_stopwords))
#org_stopwords

number of companies in companies list:  11149
number of phrases in org stopwords list:  86


In [7]:
row_idx = 10332
print(df.iloc[row_idx])#['title'])
print(len(df.iloc[row_idx]['content']))

author                                                      NaN
date                                        2020-03-02 00:00:00
domain                                           marketscreener
title         BeiGene : OVERSEAS REGULATORY ANNOUNCEMENT - F...
url           https://www.marketscreener.com/BEIGENE-LTD-261...
content       Hong Kong Exchanges and Clearing Limited and T...
topic_area                                             business
Name: 10332, dtype: object
1116259


In [8]:
# NER Tagging

#df_ner = df[df['date'].between('2020-01-01', '2020-01-31')]
df_ner = df[df['date'].between('2020-01-01', '2020-03-31')]
print('no. of articles: ',len(df_ner))
df_ner = df_ner[df_ner['content'].apply(lambda x: len(x) <= 1000000)]
df_ner = df_ner.reset_index(drop=True)
print('no. of articles after dropping long articles: ', len(df_ner)) # drop articles where text longer than 1,000,000 characters

df_ner['org_names'] = ''

for row in tqdm(range(len(df_ner))):
    pos_tagged = ner_processor(df_ner.iloc[row]['content'])
    org_names = set([X.text for X in pos_tagged.ents if X.label_ == 'ORG'])
    df_ner.at[row,'org_names'] = sorted(org_names)

# Save df after POS tagging
df_ner.to_pickle('./Data/df_ner.pickle')
#df_ner.to_csv('./data/df_ner.csv'

  0%|          | 0/48783 [00:00<?, ?it/s]

no. of articles:  48785
no. of articles after dropping long articles:  48783


  0%|          | 220/48783 [00:24<1:31:54,  8.81it/s]


KeyboardInterrupt: 

### Clean Tagged Names
- Remove leading and trailing non-letter characters e.g. ' % 2,4 ARCU Aircraft Leasing Limited' -> 'ARCU Aircraft Leasing Limited' Note: this would also drop 3M but companies with number as first letter are very rare
- Remove names that are only 1 or 2 letters
- Remove names found in org_stopwords list

In [9]:
# Clean Tagged Names

df_ner = pd.read_pickle('./Data/df_ner.pickle')
df_ner_clean = df_ner.copy()

for idx, org_names in tqdm(enumerate(df_ner['org_names'])):
    
    # remove leading and trailing non-letter characters
    org_names = set([re.sub('^[^a-zA-Z]*|[^a-zA-Z]*$','',name) for name in org_names])
    
    # remove names that are only 1 or 2 letters
    org_names = set([name.lower() for name in org_names if len(name) > 2])
    #org_names = set([name for name in org_names if len(name) > 2])
    
    # exclude names in org_stopwords list - difference between 2 sets
    org_names = org_names - set(org_stopwords) 
    df_ner_clean.at[idx,'org_names'] = sorted(org_names)
    
# Save df after POS tagging
df_ner_clean.to_pickle('./Data/df_ner_clean.pickle')

48783it [00:01, 26337.91it/s]


In [10]:
df_ner_clean = pd.read_pickle('./Data/df_ner_clean.pickle')
df_ner_clean.head(3)

,author,date,domain,title,url,content,topic_area,org_names
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[a dividend aristocrat for capital gains, a do..."
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business,"[adp, ccc, cintas, cloud-based services for em..."
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business,"[credit suisse, ford, musk, nyse, tsla, wedbus..."


In [11]:
import itertools 

# List of all org names
ner_org_names = sorted(set(list(itertools.chain(*[names for names in df_ner_clean['org_names']]))))
df_ner_org_names = pd.DataFrame(ner_org_names)
df_ner_org_names.to_pickle('./Data/df_ner_org_names.pickle')
len(ner_org_names)

133039

In [12]:
#df_ner[df_ner['org_names'].apply(lambda x: '△167 Appliance' in x)]
#df_ner_clean[df_ner_clean['org_names'].apply(lambda x: '△167 Appliance' in x)]

In [13]:
#ner_org_names[::-1]
ner_org_names[:5]

['a "bureau of diseasology',
 'a "council of state',
 'a "distinct cars note',
 'a "top pick',
 'a & ps']

In [14]:
row_idx = 837
print(df_ner.iloc[row_idx]['org_names'])
df_ner.iloc[row_idx]['content']

['British Airways', 'FTSE', 'HSBC', 'IAG', 'Markets.com', 'Prudential', 'the World Health Organisation']


'The FTSE 100 index, which had recovered on Friday after the World Health Organisation issued a measured assessment of the virus, stumbled 2.3%. The midcaps slid 2.1%, their steepest one-day decline in more than a year. All but three stocks ended in negative territory on the main bourse. News that China\'s death toll from the coronavirus discovered at the end of last year has risen to 81 dragged an index of leisure and airline stocks <.FTNMX5750> down 2.6% to its worst day in more than three and a half years. Roughly 6% has been wiped off the index since last week. The sector is exposed to a slowdown in the travel market because of the outbreak, with some standout individual losers including British Airways owner IAG, which dropped 5.4%, and China-exposed luxury brand Burberry, down 4.6%. Asia-focused financials HSBC, Prudential and Standard Chartered skidded 3.5%-5% as businesses shut shops and the Chinese government lengthened the week-long Lunar New Year holiday by three days to try

### Checking similarity of NER word against company names in company list

Rule 1 
- When the NER is only 1 word  
- The comparison is reduced to the set of common words  
- When the similarity ratio is higher than 90% it is accepted

Problems
- When a one word NER is found in multiple company names, e.g Southwest (found NER) returns company names 'Southwest Airlines' and 'Southwest Gas Corp' and 'Southwest Capital'

Rule 2
- When the NER is more than 1 word
- When part of the NER exists in the stop words list e.g. 'Capital One Financial Corp', financial and corp are words in the stop words list
- When the similarity ratio is higher than 95% it is accepted

Rule 3
- Remaining words
- Similarity ratio on best matching substring being 90% and higher


Potential Rule
- When a name is only 1 word only take the match which is the shortest length. E.g Express -> matches Amercian Express and Express Inc -> it should only take Express Inc

In [198]:
df_ner_clean['filtered_names'] = ''
df_ner_clean['filtered_names_match'] = ''
#for idx, names in tqdm(enumerate(df_ner_clean['org_names'][:50])):
for idx, names in enumerate(df_ner_clean['org_names'][800:850]):
    print('\n', idx)
    filtered_names = []
    filtered_names_match = []
    for name in names:
        for list_name in companies_list:
            # Rule 1 - NER name only 1 word -> set of common words  -> similarity ratio on set
            if len(name.split(' ')) == 1:
                ratio = fuzz.token_set_ratio(name, list_name.lower())
                if ratio >= 90:
                    print('rule1: ', name, '-', list_name, ratio)
                    filtered_names.append(name)
                    filtered_names_match.append(list_name.lower())
            # Rule 2 - NER name longer than 1 word plus part of name in stopwords list -> similarity ratio on set
            elif set(name.split(' ')).intersection(org_stopwords):
                ratio = fuzz.token_set_ratio(name, list_name.lower())
                if ratio >= 95:
                    print('rule2: ', name, '-', list_name, ratio)
                    filtered_names.append(name)
                    filtered_names_match.append(list_name)
            else:
                ratio = fuzz.partial_ratio(name, list_name.lower())
                if ratio >= 90:
                    print('rule3: ', name, '-', list_name, ratio)
                    filtered_names.append(name)
                    filtered_names_match.append(list_name)
    df_ner_clean.at[idx,'filtered_names'] = set(filtered_names)
    df_ner_clean.at[idx,'filtered_names_match'] = set(filtered_names_match)


 0
rule1:  rbc - RBC Bearings Incorp 100

 1
rule3:  morgan stanley - Morgan Stanley [Ms/Pf] 100
rule3:  morgan stanley - Morgan Stanley [Ms/Pk] 100
rule3:  morgan stanley - Morgan Stanley [Ms/Pi] 100
rule3:  morgan stanley - Morgan Stanley [Ms/Pe] 100
rule3:  morgan stanley - Morgan Stanley 4.875% Prf Perpetual USD 25 1/100 100
rule3:  morgan stanley - Morgan Stanley Dean Witter & Co. 100
rule3:  morgan stanley - Morgan Stanley Emerging Markets Debt 100
rule3:  morgan stanley - Morgan Stanley India Investment Fund Inc 100
rule3:  morgan stanley - Morgan Stanley China A Share Fund Inc 100
rule3:  morgan stanley - Morgan Stanley [Ms/Pa] 100
rule3:  morgan stanley - Morgan Stanley 100

 2
rule3:  johnson & johnson's - Johnson & Johnson 100
rule1:  mcdonald - McDonald's Corp. 100
rule1:  mcdonald - McDonald's Corp 100
rule1:  mcdonald - McDonald's Corporation 100
rule1:  starbucks - Starbucks Corp 100
rule1:  starbucks - Starbucks Corp. 100
rule3:  walt disney - The Walt Disney Company 1

In [ ]:
# Run for whole data set - Takes ~5 hours per 10,000 articles

df_ner_clean['filtered_names'] = ''
df_ner_clean['filtered_names_match'] = ''
for idx, names in tqdm(enumerate(df_ner_clean['org_names'])):
    filtered_names = []
    filtered_names_match = []
    for name in names:
        for list_name in companies_list:
            # Rule 1 - NER name only 1 word -> set of common words  -> similarity ratio on set
            if len(name.split(' ')) == 1:
                ratio = fuzz.token_set_ratio(name, list_name.lower())
                if ratio >= 90:
                    filtered_names.append(name)
                    filtered_names_match.append(list_name.lower())
            # Rule 2 - NER name longer than 1 word plus part of name in stopwords list -> similarity ratio on set
            elif set(name.split(' ')).intersection(org_stopwords):
                ratio = fuzz.token_set_ratio(name, list_name.lower())
                if ratio >= 95:
                    filtered_names.append(name)
                    filtered_names_match.append(list_name)
            else:
                ratio = fuzz.partial_ratio(name, list_name.lower())
                if ratio >= 90:
                    filtered_names.append(name)
                    filtered_names_match.append(list_name)
    if idx % 10000 == 0:
        print(f'articles to {idx} saved to file')
        df_ner_clean.to_pickle('./Data/df_ner_clean.pickle')
    df_ner_clean.at[idx,'filtered_names'] = set(filtered_names)
    df_ner_clean.at[idx,'filtered_names_match'] = set(filtered_names_match)
    
df_ner_clean.to_pickle('./Data/df_ner_clean.pickle')

0it [00:00, ?it/s]

articles to 0 saved to file


10000it [4:17:25,  1.39s/it]

articles to 10000 saved to file


20000it [8:31:09,  2.48s/it]

articles to 20000 saved to file


30000it [12:19:25,  1.15it/s]

articles to 30000 saved to file


30205it [12:24:33,  1.10s/it]

In [15]:
df_ner_clean

,author,date,domain,title,url,content,topic_area,org_names
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[a dividend aristocrat for capital gains, a do..."
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business,"[adp, ccc, cintas, cloud-based services for em..."
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business,"[credit suisse, ford, musk, nyse, tsla, wedbus..."
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech,"[cio dive, fedex, gartner, gartner's cio, h&m,..."
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer,"[cdc, chanel, climate brief, moschino, new sta..."
...,...,...,...,...,...,...,...,...
48778,Jan Lopatka,2020-03-31,reuters,"Czech virus chief defends face masks order, sa...",https://www.reuters.com/article/health-coronav...,- The Czech government’s coronavirus tsar on ...,business,"[health ministry, the world health organisatio..."
48779,Francois Murphy,2020-03-31,reuters,UPDATE 1-Austrian economy to shrink 3.2% in 'm...,https://www.reuters.com/article/austria-econom...,"By Francois Murphy VIENNA, March 31 (Reuters)...",business,"[ecb, holzmann, onb, reuters, the austrian nat..."
48780,Francois Murphy,2020-03-31,reuters,Austrian economy to shrink 3.2% in 'moderate' ...,https://www.reuters.com/article/uk-austria-eco...,Austria’s economy will shrink 3.2% this year i...,business,"[ecb, holzmann, onb, the austrian national ban..."
48781,"Michelle Conlin, Linda So, Brad Heath, Grant S...",2020-03-31,reuters,UPDATE 1-INSIGHT-Coronavirus hits hundreds of ...,https://www.reuters.com/article/health-coronav...,(Updates to include latest available figures f...,business,"[nypd, reuters, shotspotter, the dallas police..."
